# 注意事项

- 优矿数据API，晚上不能用。[优矿免费数据社区](https://uqer.io/data/browse/0/?page=1)

# 数据获取

- 获取当日分钟线： DataAPI.MktBarRTIntraDayGet

DataAPI.MktBarRTIntraDayGet(securityID=u"000001.XSHE",startTime=u"",endTime=u"",unit=u"",pandas="1")

    参数：unit—时间宽度（1/5/15/30/60分钟)
    返回值：ticker—证券代码；exchangeCD—交易所代码；totalVolume—当前bar的总交易量

In [ ]:
=if( (C3>( (sum($C$2:C3)/ COUNT($C$2:C3))+ (3*STDEV($C$2:C3)) )),
           ((sum($C$2:C3)/ COUNT($C$2:C3))+(3*STDEV($C$2:C3))) ,
      if( (C3<( (sum($C$2:C3)/ COUNT($C$2:C3))- (3*STDEV($C$2:C3)) )),
                ((sum($C$2:C3)/ COUNT($C$2:C3))- (3*STDEV($C$2:C3))), C3),C3)

In [ ]:
=IF((C3>((SUM($C$2:C3)/COUNT($C$2:C3))+(2*STDEV($C$2:C3)))),((SUM($C$2:C3)/COUNT($C$2:C3))+(2*STDEV($C$2:C3))),
 IF((C3<((SUM($C$2:C3)/COUNT($C$2:C3))-(2*STDEV($C$2:C3)))),((SUM($C$2:C3)/COUNT($C$2:C3))-(2*STDEV($C$2:C3))),C3))

### 1. 获取分钟线数据
tushare提供了一次性历史分笔数据明细：ts.get_tick_data('600848',date='2014-01-09')

**参数**

- code：股票代码，即6位数字代码
- date：日期，格式YYYY-MM-DD
- retry_count : int, 默认3,如遇网络等问题重复执行的次数
- pause : int, 默认 0,重复请求数据过程中暂停的秒数，防止请求间隔时间太短出现的问题

**返回值**
- time：时间
- price：成交价格
- change：价格变动
- volume：成交手
- amount：成交金额(元)
- type：买卖类型【买盘、卖盘、中性盘】

因此，问题就在于：（1）获取所有A股票的股票代码 | 可通过tushare接口获取；（2）获取交易所交易日日历 | 使用大盘的交易日期代替

**问题**
根据A股票股票代码列表和指定时间段的时间去获取每只股票，指定时间段的历史分钟数据。遇到的问题有：（1）新上市的股票前期没有数据(done)。（2）数据如何存储—目前考虑每只股票的分钟级数据单独存放于一个数据文件中。

In [2]:
import tushare as ts
import pandas as pd
import numpy as np
url ='E:/tangyang/tangyang/QuantitativeInvestment/data/'

ImportError: No module named 'tushare'

In [10]:
secID =ts.get_stock_basics().index.values  #(1)获取沪深上市公司基本情况

In [11]:
print secID; print len(secID) 

['300533' '300532' '603986' ..., '002809' '002807' '000155']
2915


In [26]:
trading_date =sorted(ts.get_h_data('000001',start='2015-08-18', end='2016-08-19', index=True).index.values)
#test_date =trading_date[0]
#print str(test_date)[:10]
trading_date =[str(date)[:10] for date in trading_date]
print trading_date[:10]

SyntaxError: Missing parentheses in call to 'print' (<ipython-input-26-c36f91ed8fb8>, line 5)

In [ ]:
import tushare as ts
import pandas as pd
import numpy as np


def get_min_data_by_year(secID,trading_date,url,year):
#(3)获取每只股票的历史分笔数据
    for sec in secID[40:]:
        print('-------------开始下载%s--------%d年的分钟级数据-------------'%(sec,year))
        minutes_data_per_secID =pd.DataFrame()
        for one_date in trading_date:
            print('----------%s---------'%one_date)
            minutes_data = ts.get_tick_data(code=sec,date=one_date,retry_count=4, pause=1)
            if len(minutes_data)==3:
                pass
            else:
                minutes_data.sort(inplace=True,ascending=False)
                minutes_data['date'] =len(minutes_data)*[one_date]
                print(minutes_data.head(1))
                minutes_data_per_secID =minutes_data_per_secID.append(minutes_data,ignore_index=False)
        if len(minutes_data_per_secID)==0:
            print('-------------%s只股票没有数据------------'%sec)
            pass
        else:
            print('-----------存储%s股票的数据---------'%sec)
            minutes_data_per_secID.to_csv(url+"%s_%s_min_data.csv"%(one_date[:4],sec),index=False)

def process():
    url ='D:/tangyang/study/FinancialLife/Quantitative Investment/data/'
    #(1)获取沪深上市公司基本情况
    secID =sorted(ts.get_stock_basics().index.values)  
    #(2)获取交易日日历
    for year in range(2005,2015):
        start =str(year)+'-01-01'; end =str(year)+'-12-31'
        print(start, end)
        trading_date =sorted(ts.get_h_data('000001',start=start, end=end, index=True).index.values)
        trading_date =[str(date)[:10] for date in trading_date] 
        get_min_data_by_year(secID,trading_date,url,year)
process()

2005-01-01 2005-12-31
[Getting data:]###-------------开始下载000048--------2005年的分钟级数据-------------
----------2005-01-04---------


D:\software\installation\Anaconda\Lib\site-packages\ipykernel\__main__.py:17: FutureWarning: sort(....) is deprecated, use sort_index(.....)


        time  price change  volume  amount type        date
45  09:30:08   2.37   2.37       7    1659   买盘  2005-01-04
----------2005-01-05---------
         time  price change  volume  amount type        date
102  09:36:27   2.35   2.35       5    1175   买盘  2005-01-05
----------2005-01-06---------
        time  price change  volume  amount type        date
65  10:03:05   2.32   2.32      24    5568   买盘  2005-01-06
----------2005-01-07---------
         time  price change  volume  amount type        date
114  09:33:53   2.34   2.34      59   13806   买盘  2005-01-07
----------2005-01-10---------
         time  price change  volume  amount type        date
114  09:40:13   2.39   2.39      13    3107   买盘  2005-01-10
----------2005-01-11---------
         time  price change  volume  amount type        date
105  09:43:46   2.41   2.41      33    7953   买盘  2005-01-11
----------2005-01-12---------
        time  price change  volume  amount type        date
95  09:34:20   2.47   2.47      

In [13]:
df = ts.get_tick_data('000001',date='2014-01-09')

In [ ]:
TEST =ts.get_tick_data(code='000001',date='2005-09-28',retry_count=4, pause=1)
print(TEST.head())

In [3]:
import os,sys


In [4]:
print(os.getcwd())

C:\Users\tangy\Documents\tangyang


In [56]:
5*['2016']
date_r =pd.date_range('2001-01-01','2016-08-19', freq='D')
print(date_r)
#'01-01','12-31'
for i in range(2001,2016):
    print(str(i))

DatetimeIndex(['2001-01-01', '2001-01-02', '2001-01-03', '2001-01-04',
               '2001-01-05', '2001-01-06', '2001-01-07', '2001-01-08',
               '2001-01-09', '2001-01-10',
               ...
               '2016-08-10', '2016-08-11', '2016-08-12', '2016-08-13',
               '2016-08-14', '2016-08-15', '2016-08-16', '2016-08-17',
               '2016-08-18', '2016-08-19'],
              dtype='datetime64[ns]', length=5710, freq='D')
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015


In [18]:
import numpy as np
print(str(np.array(1))+','+str(np.array(4))+','+str(np.array(9)))


1,4,9


In [1]:
import pandas as pd

## 明天游泳

- 早上
- 中午
- 晚上

In [ ]:
d